<a href="https://colab.research.google.com/github/anshupandey/prompt_engineering/blob/main/code2_Prompt_Engineering_with_OpenAI.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Prompt Engineering Techniques


Prompt engineering is the process of designing and refining prompts for large language models (LLMs) to elicit desired responses. Different techniques exist to improve the model's performance and controllability.

In [1]:
!pip install openai --quiet

In [2]:
import os
os.environ['OPENAI_API_KEY'] = 'sk-proj-HwsAjVFo6hk-xxxxxxx-xxxxxxxxx-xxxxxx-xxxxxxxx'

In [3]:
from openai import OpenAI
client = OpenAI()

In [4]:
# creating a function to get outcome
def generate_response(prompt,model="gpt-4o",temperature=0):
  messages = [{"role":"user","content":prompt}]
  response = client.responses.create(
      model = model,
      input = messages,
      temperature=temperature
  )
  return response.output_text


In [5]:
generate_response("Hi! How are you.")

"Hi! I'm doing well, thank you. How about you?"


## Zero-Shot Prompting
**Description:** In zero-shot prompting, the model is given a prompt with a task and no examples of how to perform it. The model is expected to perform the task based solely on its pre-training.

**Application Areas:** Simple tasks where the model has a strong understanding from its training data, such as classification, translation, or summarization for common domains.

**When to Use:**
- When the task is straightforward and aligns well with the model's pre-training.
- When collecting examples for few-shot prompting is difficult or time-consuming.
- As a baseline to see how well the model performs without explicit examples.

**Example:**
Prompt: "Translate the following English sentence to French: 'Hello, how are you?'"



In [6]:
# zero shot prompting
prompt = f"""
Create a conversation story between child and grandparent.
"""
response = generate_response(prompt)
print(response)

**Title: A Walk Down Memory Lane**

**Setting:** A cozy living room with a crackling fireplace. The child, Lily, is sitting on the floor, playing with a puzzle, while her Grandpa Joe relaxes in an armchair, sipping tea.

**Lily:** Grandpa, can you tell me a story from when you were little?

**Grandpa Joe:** Oh, I have plenty of stories! Let me think... Ah, how about the time I built a treehouse with my friends?

**Lily:** A treehouse? That sounds amazing! Was it big?

**Grandpa Joe:** It felt enormous to us back then. We used old wooden planks and ropes. It was our secret hideout.

**Lily:** Did you have a secret password to get in?

**Grandpa Joe:** We sure did! It was "Blueberry Pie." We thought it was the most clever password ever.

**Lily:** (giggling) That's funny! What did you do in the treehouse?

**Grandpa Joe:** We’d pretend to be pirates searching for treasure. Sometimes, we’d just sit and watch the clouds, making up stories about their shapes.

**Lily:** I wish I could have 

In [23]:
prompt = """Classify the customer's request into one of the following categories:
- Account Balance Inquiry
- Loan Application Status
- Report Lost Card
- Fund Transfer Issue
- Credit Card Application


Customer: "Can you tell me what to do in case I misplaced my debit card?"
Category:"""
output = generate_response(prompt)
print(output)

Report Lost Card


## Few-Shot Prompting
**Description:** Few-shot prompting involves providing the model with a few examples (typically 1 to 5) of the desired input-output pairs before presenting the actual task. This helps the model understand the format and type of output expected.

**Application Areas:** Tasks where the desired output format is specific, or the task requires understanding a particular style or pattern. Examples include generating text in a specific style, answering questions in a particular format, or performing tasks on less common data.

**When to Use:**
- When zero-shot performance is not sufficient.
- When the task requires a specific output format or style.
- When the task involves domains or data less frequently seen during pre-training.
- To guide the model towards desired behavior without fine-tuning.
**Example:**
Prompt:
"Translate to Spanish:
English: 'Goodbye'
Spanish: 'Adiós'
English: 'Thank you'
Spanish: 'Gracias'
English: 'Please'
Spanish: 'Por favor'
English: 'Excuse me'
Spanish: 'Disculpe'"


In [7]:
# few shot prompting
prompt = f"""
your task is to answer in a consistent sytle:

<child>: Teach me about patience.
<grandma>: Patience is like waiting for whole day to see the moon in the evening and then sleeping after having a look at it.

<child>: Teach me about resilience.
"""
response = generate_response(prompt)
print(response)

<grandma>: Resilience is like a tree that bends in the wind but never breaks, standing tall again when the storm has passed.


#### Few shot prompting with role setup

In [12]:
result = client.responses.create(
    model='gpt-4o',  # e.g. gpt-35-instant
    max_output_tokens=200,
    temperature=0,
    input=[
        {"role": "system", "content": "You are a helpful assistant."},
        {"role": "user", "content": "That was an awesome experience"},
        {"role": "assistant", "content": "positive"},
        {"role": "user", "content": "I won't do that again"},
        {"role": "assistant", "content": "negative"},
        {"role": "user", "content": "That was not worth my time"},
        {"role": "assistant", "content": "negative"},
        {"role": "user", "content": "You can't miss this"}
    ],
)
print(result.output_text)

positive


In [14]:
prompt = """Classify the customer's request into one of the following categories:
- Account Balance Inquiry
- Loan Application Status
- Report Lost Card
- Fund Transfer Issue
- Credit Card Application

Examples:
Customer: "What's the current balance in my savings account?"
Category: Account Balance Inquiry

Customer: "I need to know where my loan application stands."
Category: Loan Application Status

Customer: "My credit card is missing, I think I lost it."
Category: Report Lost Card

Customer: "I tried to send money to my friend but it's not going through."
Category: Fund Transfer Issue

Customer: "I want to apply for a new credit card."
Category: Credit Card Application

Customer: "Can you tell me how much money I have in my checking account?"
Category:"""

output = generate_response(prompt)
print(output)

Account Balance Inquiry


In [15]:
def get_categorization(qus):
    prompt = f"""Classify the customer's request into one of the following categories:
      - Account Balance Inquiry
      - Loan Application Status
      - Report Lost Card
      - Fund Transfer Issue
      - Credit Card Application

      Examples:
      Customer: "What's the current balance in my savings account?"
      Category: Account Balance Inquiry

      Customer: "I need to know where my loan application stands."
      Category: Loan Application Status

      Customer: "My credit card is missing, I think I lost it."
      Category: Report Lost Card

      Customer: "I tried to send money to my friend but it's not going through."
      Category: Fund Transfer Issue

      Customer: "I want to apply for a new credit card."
      Category: Credit Card Application

      Customer: {qus}
      Category:"""
    output = generate_response(prompt)
    return output


In [16]:
get_categorization("can you help me with my checkin balance?")

'Account Balance Inquiry'

In [17]:
get_categorization("I think I missed my credit card while travelling yesterday at airport, what do i do next?")

'Report Lost Card'

In [18]:
get_categorization("How do apply for a new credit card?")

'Credit Card Application'


## Chain-of-Thought (CoT) Prompting
**Description:** CoT prompting involves explicitly asking the model to show its reasoning steps or thought process before arriving at the final answer. This is often achieved by including phrases like "Let's think step by step" in the prompt. CoT can be used in a zero-shot or few-shot setting (Zero-shot CoT or Few-shot CoT).

**Application Areas:** Complex reasoning tasks such as arithmetic, logical deduction, multi-step problem-solving, and question answering that requires synthesis of information.

**When to Use:**
- When the task requires multi-step reasoning.
- When the final answer depends on intermediate steps.
- To improve the accuracy on complex problems by guiding the model through the solution process.
- To understand the model's reasoning and debug incorrect outputs.
**Example (Few-shot CoT):**
Prompt:
"Q: Roger has 5 tennis balls. He buys 2 more cans of tennis balls. Each can has 3 tennis balls. How many tennis balls does he have now?
A: Roger started with 5 balls. He bought 2 cans with 3 balls each, so he bought 2 * 3 = 6 balls. Total balls = 5 + 6 = 11. The answer is 11.

Q: The cafeteria had 23 apples. If they used 20 and then bought 6 more, how many apples do they have?
A: Let's think step by step. The cafeteria had 23 apples. They used 20, so they had 23 - 20 = 3 apples left. Then they bought 6 more, so they have 3 + 6 = 9 apples. The answer is 9.

Q: [New complex question]
A: Let's think step by step."


In [20]:
# Without Chain of Thought Prompting = Only Few Shot (No CoT)
prompt = f"""
Answer below question
Teacher: Johan had 5 apples, he bought 4 more boxes with 5 apples each. How many apples does he have now?
Student: 25 apples

Teacher: Marry had 5 pens, she gave 3 to her friend. and bought 2 boxes of pen with 3 pen in each box. How many pens does she have now?
Student:
"""
response = generate_response(prompt)
print(response)

To solve the problem:

1. Marry initially had 5 pens.
2. She gave 3 pens to her friend, so she has \(5 - 3 = 2\) pens left.
3. She bought 2 boxes with 3 pens in each box, which is \(2 \times 3 = 6\) pens.

Adding these together, Marry has \(2 + 6 = 8\) pens now.


In [21]:
# with Chain of Thought prompting = Few Shot CoT
prompt = f"""

Teacher: Johan had 5 apples, he bought 4 more boxes with 5 apples each. How many apples does he have now?
Student: Johan had 5 apples, 4 boxes with 5 apples each = 5 + 4*5 = 25 apples

Teacher: Marry had 5 pens, she gave 3 to her friend. and bought 2 boxes of pen with 3 pen in each box. How many pens does she have now?
Student:

"""
response = generate_response(prompt)
print(response)

Marry had 5 pens, gave 3 to her friend, and bought 2 boxes with 3 pens each.

Starting pens: 5  
Pens given away: 3  
Remaining pens: 5 - 3 = 2  

Pens bought: 2 boxes × 3 pens each = 6 pens  

Total pens now: 2 + 6 = 8 pens  

So, Marry has 8 pens now.


In [22]:
# with Chain of Thought prompting = Zero shot CoT
prompt = f"""

Think step by step to answer below question, do not come to conclusion without following process.
Teacher: Marry had 5 pens, she gave 3 to her friend. and bought 2 boxes of pen with 3 pen in each box. How many pens does she have now?
Student:

"""
response = generate_response(prompt)
print(response)

Let's break down the problem step by step:

1. **Initial Pens:**
   - Marry starts with 5 pens.

2. **Pens Given Away:**
   - She gives 3 pens to her friend.
   - Calculation: \(5 - 3 = 2\) pens remaining.

3. **Pens Bought:**
   - She buys 2 boxes of pens, with 3 pens in each box.
   - Total pens bought: \(2 \times 3 = 6\) pens.

4. **Total Pens Now:**
   - Add the pens she has after giving some away to the pens she bought.
   - Calculation: \(2 + 6 = 8\) pens.

So, Marry has 8 pens now.


In [28]:
# without Chain of thought

prompt = """
Based on the following eligibility criteria, determine which tier of the Premium Checking Account (Gold, Platinum, or None) the customer qualifies for.
Answer the question:
**Gold Tier Eligibility:**
- Maintain a minimum average daily balance of $5,000 OR
- Have at least $25,000 in combined linked accounts.
- Must also have at least 5 debit card transactions per month.

**Platinum Tier Eligibility:**
- Maintain a minimum average daily balance of $25,000 OR
- Have at least $100,000 in combined linked accounts.
- Must also have at least 10 debit card transactions per month.
- Additionally, must have a credit score of 700 or higher.

**Customer Profile:**
- Average daily checking balance: $8,000
- Combined linked accounts balance: $45,000
- Debit card transactions last month: 17
- Credit score: 720

Which tier does this customer qualify for?
"""
print(generate_response(prompt))

The customer qualifies for the **Gold Tier**.

**Gold Tier Requirements:**
- Minimum average daily balance of $5,000 OR $25,000 in combined linked accounts: The customer has an average daily balance of $8,000, which meets this requirement.
- At least 5 debit card transactions per month: The customer has 17 transactions, which meets this requirement.

**Platinum Tier Requirements:**
- Minimum average daily balance of $25,000 OR $100,000 in combined linked accounts: The customer does not meet this requirement, as they have $8,000 in average daily balance and $45,000 in combined linked accounts.
- At least 10 debit card transactions per month: The customer meets this requirement with 17 transactions.
- Credit score of 700 or higher: The customer meets this requirement with a credit score of 720.

Since the customer does not meet the balance requirement for the Platinum Tier, they qualify for the Gold Tier.


In [27]:
# Zero shot Chain of thought

prompt = """
Based on the following eligibility criteria, determine which tier of the Premium Checking Account (Gold, Platinum, or None) the customer qualifies for.
Answer the question:
**Gold Tier Eligibility:**
- Maintain a minimum average daily balance of $5,000 OR
- Have at least $25,000 in combined linked accounts.
- Must also have at least 5 debit card transactions per month.

**Platinum Tier Eligibility:**
- Maintain a minimum average daily balance of $25,000 OR
- Have at least $100,000 in combined linked accounts.
- Must also have at least 10 debit card transactions per month.
- Additionally, must have a credit score of 700 or higher.

**Customer Profile:**
- Average daily checking balance: $8,000
- Combined linked accounts balance: $15,000
- Debit card transactions last month: 7
- Credit score: 720

Which tier does this customer qualify for?
Lets analyze the customer profile step by step and try answer the question.
"""
print(generate_response(prompt))

Let's analyze the customer profile step by step:

**Gold Tier Eligibility:**
- **Average Daily Balance:** The customer has $8,000, which is above the $5,000 requirement.
- **Combined Linked Accounts:** The customer has $15,000, which is below the $25,000 requirement.
- **Debit Card Transactions:** The customer has 7 transactions, which meets the requirement of at least 5.

The customer meets all the criteria for the Gold Tier.

**Platinum Tier Eligibility:**
- **Average Daily Balance:** The customer has $8,000, which is below the $25,000 requirement.
- **Combined Linked Accounts:** The customer has $15,000, which is below the $100,000 requirement.
- **Debit Card Transactions:** The customer has 7 transactions, which is below the 10 required.
- **Credit Score:** The customer has a score of 720, which meets the requirement of 700 or higher.

The customer does not meet the criteria for the Platinum Tier.

**Conclusion:**
The customer qualifies for the **Gold Tier**.


In [29]:
# Zero shot Chain of thought

prompt = """
Based on the following eligibility criteria, determine which tier of the Premium Checking Account (Gold, Platinum, or None) the customer qualifies for.
Answer the question:
**Gold Tier Eligibility:**
- Maintain a minimum average daily balance of $5,000 OR
- Have at least $25,000 in combined linked accounts.
- Must also have at least 5 debit card transactions per month.

**Platinum Tier Eligibility:**
- Maintain a minimum average daily balance of $25,000 OR
- Have at least $100,000 in combined linked accounts.
- Must also have at least 10 debit card transactions per month.
- Additionally, must have a credit score of 700 or higher.

---

**Example 1:**
**Customer Profile:**
- Average daily checking balance: $4,000
- Combined linked accounts balance: $30,000
- Debit card transactions last month: 6
- Credit score: 650

**Let's analyze the customer's eligibility for each tier step by step:**

**1. Evaluate Gold Tier Eligibility:**
    * **Condition 1 (Financial - OR):**
        * Average daily balance: $4,000. Is this >= $5,000? No.
        * Combined linked accounts: $30,000. Is this >= $25,000? Yes.
        * Since one of the OR conditions is met (combined linked accounts is $30,000), the financial requirement for Gold is met.
    * **Condition 2 (Debit Card Transactions - AND):**
        * Debit card transactions: 6. Is this >= 5? Yes.
    * **Conclusion for Gold Tier:** Both the financial (OR) condition and the debit card transaction (AND) condition are met.
    * **Interim Result:** Customer is eligible for Gold Tier.

**2. Evaluate Platinum Tier Eligibility:**
    * **Condition 1 (Financial - OR):**
        * Average daily balance: $4,000. Is this >= $25,000? No.
        * Combined linked accounts: $30,000. Is this >= $100,000? No.
        * Neither of the OR conditions is met.
    * **Condition 2 (Debit Card Transactions - AND):**
        * Debit card transactions: 6. Is this >= 10? No.
    * **Condition 3 (Credit Score - AND):**
        * Credit score: 650. Is this >= 700? No.
    * **Conclusion for Platinum Tier:** None of the key criteria (financial, transactions, credit score) are met.
    * **Interim Result:** Customer is NOT eligible for Platinum Tier.

**Final Determination:**
Based on the step-by-step evaluation, the customer meets all the criteria for the **Gold Tier** but fails to meet the criteria for the Platinum Tier.
**Therefore, the customer qualifies for the Gold Tier.**

---

**Example 2:**
**Customer Profile:**
- Average daily checking balance: $30,000
- Combined linked accounts balance: $120,000
- Debit card transactions last month: 12
- Credit score: 750

**Let's analyze the customer's eligibility for each tier step by step:**

**1. Evaluate Gold Tier Eligibility:**
    * **Condition 1 (Financial - OR):**
        * Average daily balance: $30,000. Is this >= $5,000? Yes.
        * Combined linked accounts: $120,000. Is this >= $25,000? Yes.
        * One of the OR conditions is met (both are, but only one needed). Financial requirement for Gold is met.
    * **Condition 2 (Debit Card Transactions - AND):**
        * Debit card transactions: 12. Is this >= 5? Yes.
    * **Conclusion for Gold Tier:** Both conditions are met.
    * **Interim Result:** Customer is eligible for Gold Tier.

**2. Evaluate Platinum Tier Eligibility:**
    * **Condition 1 (Financial - OR):**
        * Average daily balance: $30,000. Is this >= $25,000? Yes.
        * Combined linked accounts: $120,000. Is this >= $100,000? Yes.
        * One of the OR conditions is met (both are). Financial requirement for Platinum is met.
    * **Condition 2 (Debit Card Transactions - AND):**
        * Debit card transactions: 12. Is this >= 10? Yes.
    * **Condition 3 (Credit Score - AND):**
        * Credit score: 750. Is this >= 700? Yes.
    * **Conclusion for Platinum Tier:** All three conditions (financial, transactions, credit score) are met.
    * **Interim Result:** Customer is eligible for Platinum Tier.

**Final Determination:**
Based on the step-by-step evaluation, the customer meets all the criteria for both Gold and Platinum tiers. In such cases, the highest qualifying tier is typically granted.
**Therefore, the customer qualifies for the Platinum Tier.**

**Customer Profile:**
- Average daily checking balance: $8,000
- Combined linked accounts balance: $15,000
- Debit card transactions last month: 7
- Credit score: 720

Which tier does this customer qualify for?
Lets analyze the customer profile step by step and try answer the question.
"""
print(generate_response(prompt))

**1. Evaluate Gold Tier Eligibility:**

* **Condition 1 (Financial - OR):**
  * Average daily balance: $8,000. Is this >= $5,000? Yes.
  * Combined linked accounts: $15,000. Is this >= $25,000? No.
  * One of the OR conditions is met (average daily balance is $8,000). Financial requirement for Gold is met.

* **Condition 2 (Debit Card Transactions - AND):**
  * Debit card transactions: 7. Is this >= 5? Yes.

* **Conclusion for Gold Tier:** Both the financial (OR) condition and the debit card transaction (AND) condition are met.
* **Interim Result:** Customer is eligible for Gold Tier.

**2. Evaluate Platinum Tier Eligibility:**

* **Condition 1 (Financial - OR):**
  * Average daily balance: $8,000. Is this >= $25,000? No.
  * Combined linked accounts: $15,000. Is this >= $100,000? No.
  * Neither of the OR conditions is met.

* **Condition 2 (Debit Card Transactions - AND):**
  * Debit card transactions: 7. Is this >= 10? No.

* **Condition 3 (Credit Score - AND):**
  * Credit score: 7

## Tree of Thoughts (ToT)

Tree of Thoughts (ToT) prompting is a framework that generalizes over chain-of-thought prompting and encourages exploration over thoughts that serve as intermediate steps for general problem-solving with language models.



**Description:** ToT is an extension of CoT where the model explores multiple reasoning paths (a tree structure) instead of a single linear chain of thought. It generates multiple intermediate thoughts and explores their consequences, allowing for backtracking and exploration of different possibilities before selecting the most promising path to the solution.

**Application Areas:** More complex and open-ended reasoning tasks than CoT, such as creative writing, strategic planning, and problem-solving that might require exploring multiple hypotheses.

**When to Use:**
- When the problem has multiple potential approaches or intermediate steps.
- When a single linear chain of thought might lead to local optima or dead ends.
- For tasks requiring more extensive exploration of possibilities.

**How it Works (conceptually):** ToT prompting breaks problems down into smaller parts, similar to CoT prompting, but goes further by combining this with the ability to explore multiple solution paths in parallel, forming a tree instead of a single chain. Each thought is generated or solved independently and passed to the next step, allowing the model to self-evaluate and decide whether to continue with that path or choose another.

Generate potential intermediate thoughts -> Evaluate their validity/promise -> Select the most promising ones -> Recursively generate further thoughts until a solution is reached. This typically requires more sophisticated prompting or external control mechanisms than basic CoT.


In [30]:

prompt = """
Imagine 5 different experts are answering this question.
They will brainstorm the answer step by step reasoning carefully and taking all facts into consideration
All experts will write down 1 step of their thinking,
then share it with the group.
They will each critique their response, and the all the responses of others
They will check their answer based on science and the laws of physics
They will keep going through steps until they reach their conclusion taking into account the thoughts of the other experts
If at any time they realise that there is a flaw in their logic they will backtrack to where that flaw occurred
If any expert realises they're wrong at any point then they acknowledges this and start another train of thought
Each expert will assign a likelihood of their current assertion being correct
Continue until the experts agree on the single most likely location
The question is.
1. Carlos is at the swimming pool.
2. He walks to the locker room, carrying a towel.
3. He puts his watch in the towel and carries the towel tightly to a lounger at the poolside.
4. At the lounger he opens and vigorously shakes the towel, then walks to the snack bar.
5. He leaves the towel at the snack bar, then walks to the diving board.
6. Later Carlos realises he has has lost his watch. Where is the single most likely location of the watch?
"""


ans=generate_response(prompt)

print(ans)

**Expert 1: Initial Observation**

Step 1: Carlos placed his watch in the towel and carried it to the lounger. The vigorous shaking of the towel at the lounger is a critical point where the watch could have fallen out.

Likelihood: 70%

**Expert 2: Consideration of Movement**

Step 1: Carlos moved from the locker room to the lounger, then to the snack bar, and finally to the diving board. The watch could have fallen out at any of these points, but the vigorous shaking at the lounger is the most likely.

Likelihood: 75%

**Expert 3: Analysis of Actions**

Step 1: The act of vigorously shaking the towel suggests a high chance of the watch falling out at the lounger. The towel was left at the snack bar, but no shaking occurred there.

Likelihood: 80%

**Expert 4: Physics and Probability**

Step 1: Considering the laws of physics, the force applied during the shaking at the lounger increases the probability of the watch falling out there compared to other locations.

Likelihood: 85%

**Exp

In [32]:

# ToT Prompt for Loan Application Approval

prompt = """
Imagine 5 different loan officers are reviewing a customer's loan application.
They will brainstorm the decision step by step, reasoning carefully and taking all facts into consideration from the application.
Each loan officer will write down 1 key observation or analysis step from the application data,
then share it with the group.
They will each critique their response, and all the responses of others based on standard lending practices and risk assessment criteria.
They will check their analysis against established bank policies and economic indicators.
They will keep going through steps until they reach their conclusion on whether to approve or reject the loan, taking into account the observations and critiques of the other loan officers.
If at any time they realize there is a flaw in their analysis or a crucial piece of information is overlooked, they will backtrack to where that flaw occurred.
If any loan officer realizes their assessment is weak or incorrect at any point, they acknowledge this and start another train of thought, focusing on different aspects of the application.
Each loan officer will assign a likelihood (e.g., High Risk, Medium Risk, Low Risk) based on their current assertion about the application's overall risk profile.
Continue until the loan officers agree on the single most likely outcome (Approve or Reject) based on a consensus risk assessment.

The customer's loan application details are as follows:

Customer Name: [Customer Name]
Loan Amount Requested: [Amount]
Loan Type: [e.g., Personal Loan, Mortgage]
Annual Income: [Income]
Credit Score: [Score]
Employment Status: [e.g., Full-time, Self-employed]
Employment Duration: [Duration]
Existing Debt Obligations: [List or summary]
Assets: [List or summary]
Purpose of Loan: [Purpose]
Payment History: [Summary of past credit behavior]

Based on these details, where is the single most likely outcome of the loan application (Approve or Reject)?
"""

# Replace bracketed information with actual application details
customer_details_prompt = prompt.replace("[Customer Name]", "John Doe") \
                               .replace("[Amount]", "$50,000") \
                               .replace("[Loan Type]", "Personal Loan") \
                               .replace("[Income]", "$75,000") \
                               .replace("[Score]", "720") \
                               .replace("[Employment Status]", "Full-time") \
                               .replace("[Employment Duration]", "5 years") \
                               .replace("[Existing Debt Obligations]", "Credit card balance: $5,000, Auto loan payment: $400/month") \
                               .replace("[Assets]", "Savings account: $20,000, Car value: $15,000") \
                               .replace("[Purpose of Loan]", "Home renovation") \
                               .replace("[Summary of past credit behavior]", "No late payments in the last 3 years")

ans = generate_response(customer_details_prompt)

print(ans)


To determine the outcome of John Doe's loan application, let's go through the process step by step with the loan officers' observations and analyses.

### Step 1: Initial Observations

**Loan Officer 1:**
- **Observation:** John Doe has a credit score of 720, which is considered good.
- **Critique:** A good credit score suggests responsible credit behavior, but we need to consider other factors like income and debt obligations.
- **Risk Assessment:** Low Risk

**Loan Officer 2:**
- **Observation:** The requested loan amount is $50,000, and his annual income is $75,000.
- **Critique:** The debt-to-income ratio needs to be calculated to assess affordability.
- **Risk Assessment:** Medium Risk

**Loan Officer 3:**
- **Observation:** John has no late payments in the last 3 years.
- **Critique:** This indicates a strong payment history, which is a positive factor.
- **Risk Assessment:** Low Risk

**Loan Officer 4:**
- **Observation:** Employment status and duration are missing.
- **Critique

In [33]:
# ToT Prompt for Enterprise Banking Scenario: Supply Chain Finance Optimization

prompt = """
Imagine 5 different experts in Supply Chain Finance, Risk Management, Data Science, Operations, and Customer Relationship Management are collaborating to address the following challenge:

A large manufacturing company, a client of your bank, is experiencing significant working capital strain due to long payment terms with their suppliers (averaging 90 days)
and relatively short payment terms from their buyers (averaging 45 days).
This creates a cash flow gap that they currently bridge using expensive short-term loans from other institutions.

Your bank wants to propose a comprehensive Supply Chain Finance (SCF) solution that benefits both the manufacturing company (the buyer) and their key suppliers,
 increases the bank's revenue through new products, and strengthens the overall client relationship.
 The goal is not just to offer a standard factoring or reverse factoring product, but to design an integrated, optimized solution leveraging data and
 potentially incorporating risk mitigation strategies for the bank.

The experts will brainstorm the design of this SCF solution step by step, reasoning carefully and taking all relevant factors into consideration for the buyer,
their suppliers, and the bank.

All experts will write down 1 step of their thinking related to designing this optimized SCF solution, then share it with the group.
They will each critique their response, and all the responses of others.
They will consider the perspectives and constraints of the buyer (manufacturing company), their diverse set of suppliers (ranging from large corporations to small businesses), and the bank (risk appetite, operational capacity, revenue goals).
They will keep going through steps until they reach a well-defined and compelling SCF solution proposal, taking into account the thoughts of the other experts.
If at any time they realize that there is a flaw in their logic or a critical aspect is missed, they will backtrack to where that flaw occurred.
If any expert realizes they're wrong at any point, they acknowledge this and start another train of thought.
Each expert will assign a likelihood of their current proposed element of the solution being successful or viable.

Continue until the experts agree on the single most effective and viable integrated SCF solution proposal that addresses the working capital needs of the buyer and their suppliers while creating a valuable new revenue stream and strengthening the relationship for the bank.

The challenge is:
Design an optimized and integrated Supply Chain Finance solution for a large manufacturing client facing a working capital gap due to mismatched payment terms, considering the needs of their suppliers and the bank's objectives.
"""

ans = generate_response(prompt)
print(ans)


**Step 1: Supply Chain Finance Expert**

**Proposal:**
Introduce a dynamic discounting platform that allows the manufacturing company to pay suppliers early in exchange for a discount. This platform will be integrated with the bank's systems to offer financing options to the manufacturing company, enabling them to pay suppliers early without straining their cash flow.

**Critique:**
- **Pros:** Provides immediate cash flow relief to suppliers, strengthens supplier relationships, and offers the bank a new revenue stream through financing fees.
- **Cons:** Relies on the manufacturing company's willingness to pay early and suppliers' willingness to accept discounts.
- **Likelihood of Success:** 70%

**Step 2: Risk Management Expert**

**Proposal:**
Implement a credit risk assessment tool to evaluate the financial health of suppliers and the manufacturing company. This tool will help the bank set appropriate credit limits and interest rates for financing.

**Critique:**
- **Pros:** Mitigat

## Chain-of-Verification (CoVe) Prompting

**Description:** Chain-of-Verification (CoVe) is a prompting technique designed to improve the factual accuracy of Large Language Models (LLMs) by explicitly prompting the model to generate a draft response, then generate verification questions for its own response, answer those questions independently of the initial draft, and finally use the answers to revise and improve the original draft. This creates a self-correction mechanism.

**Application Areas:** Tasks requiring high factual accuracy and reduced hallucination, such as question answering, summarization of factual information, and generating reports or summaries based on provided text. It is particularly useful when dealing with complex or potentially ambiguous information where the model might otherwise confidently generate incorrect details.

**When to Use:**
- When factual accuracy is critical and errors could be harmful or misleading.
- To reduce hallucinations and improve the reliability of the model's output.
- When verifying information against external sources is not immediately feasible or desired within the prompting process itself.
- For generating summaries or answers that require synthesizing information from multiple potential points within the model's knowledge or context.

**How it Works:**
1. **Generate Initial Draft:** The model first generates a direct answer or response to the original prompt.
2. **Generate Verification Questions:** The model is then prompted to generate questions that would help verify the correctness of the statements made in the initial draft.
3. **Answer Verification Questions:** The model answers these verification questions *independently* of the initial draft, drawing from its internal knowledge or provided context. This step is crucial as it prevents the model from simply confirming its own potential errors.
4. **Revise Initial Draft:** The model compares the answers to the verification questions with the original draft and revises the draft to align with the verified information.

This iterative process helps the model identify inconsistencies or inaccuracies in its initial output and correct them before presenting the final response.

**Example (Conceptual):**

Prompt: "Explain the process of photosynthesis and its main outputs."

1.  **Initial Draft (Internal):** "Photosynthesis is how plants make food. They use sunlight, water, and carbon dioxide. The main output is oxygen and sugar." (Might contain slight inaccuracies or missing details).
2.  **Generate Verification Questions (Internal):**
    *   "What are the inputs of photosynthesis?"
    *   "What are the outputs of photosynthesis?"
    *   "What is the role of chlorophyll?"
    *   "Where does photosynthesis primarily occur in a plant?"
3.  **Answer Verification Questions (Internal, independently):**
    *   "Inputs are carbon dioxide, water, and light energy."
    *   "Outputs are glucose (a sugar) and oxygen."
    *   "Chlorophyll is a pigment that absorbs light energy."
    *   "It occurs mainly in chloroplasts within plant cells."
4.  **Revise Initial Draft (Internal, based on verified answers):** "Photosynthesis is the process by which green plants and some other organisms use sunlight to synthesize foods with the help of chlorophyll. During photosynthesis, plants take in carbon dioxide from the air, water from the soil, and light energy from the sun. Using the energy absorbed by chlorophyll in their chloroplasts, they convert these inputs into glucose, a sugar they use for energy and growth, and oxygen, which is released as a byproduct."

The final output presented to the user would be the revised draft. While the user doesn't see the intermediate steps, the process happens internally within the model's processing based on the CoVe prompt structure.


In [34]:
Question="Name some athletes who were born in United states"

prompt=f'''Here is the question: {Question}.

First, generate a response.

Then, create and answer verification questions based on this response to check for accuracy.
Think it through and make sure you are extremely accurate based on the question asked.

After answering each verification question, consider these answers and revise the initial response to formulate a final, verified answer.
Ensure the final response reflects the accuracy and findings from the verification process.'''



ans=generate_response(prompt)

print(ans)


**Initial Response:**

Here are some athletes who were born in the United States:

1. Michael Jordan
2. Serena Williams
3. Tom Brady
4. Simone Biles
5. LeBron James

**Verification Questions and Answers:**

1. **Was Michael Jordan born in the United States?**
   - Yes, Michael Jordan was born in Brooklyn, New York.

2. **Was Serena Williams born in the United States?**
   - Yes, Serena Williams was born in Saginaw, Michigan.

3. **Was Tom Brady born in the United States?**
   - Yes, Tom Brady was born in San Mateo, California.

4. **Was Simone Biles born in the United States?**
   - Yes, Simone Biles was born in Columbus, Ohio.

5. **Was LeBron James born in the United States?**
   - Yes, LeBron James was born in Akron, Ohio.

**Revised Final Response:**

Here are some athletes who were born in the United States:

1. Michael Jordan (Brooklyn, New York)
2. Serena Williams (Saginaw, Michigan)
3. Tom Brady (San Mateo, California)
4. Simone Biles (Columbus, Ohio)
5. LeBron James (Akron, Oh

In [35]:
prompt = """
In an enterprise banking context, a customer is requesting a complex derivative trade.
This involves intricate details regarding the underlying asset, notional value, tenor, strike price, option type (if applicable),
counterparty credit risk, regulatory compliance, and internal risk limits.

The task is to assess the viability and risks of this specific derivative trade request for the bank.

First, generate an initial assessment of the trade request, including potential benefits and immediate red flags based on standard derivative trading practices
and market knowledge.

Then, create and answer a series of verification questions specifically tailored to validate the critical details and associated risks mentioned in the initial assessment.
 Think through each question carefully, drawing upon knowledge of financial instruments, risk management frameworks, regulatory requirements (e.g., Dodd-Frank, EMIR),
 and internal banking policies.

Examples of potential verification questions:
- Is the counterparty's credit risk within acceptable limits for a trade of this size and tenor?
- Does this trade comply with all relevant regulatory requirements for derivative transactions?
- Does the pricing of this derivative accurately reflect current market conditions and volatility?
- How does this trade impact the bank's overall exposure to the underlying asset?
- Are the operational processes and systems in place to handle the settlement and ongoing management of this specific derivative type?
- Has the client undergone the necessary suitability and appropriateness checks for complex products like this?

After answering each verification question independently, consider these answers and revise the initial assessment to formulate a final,
 verified decision on whether to proceed with the trade, including any necessary conditions or risk mitigation steps.
 Ensure the final response is a comprehensive analysis reflecting the accuracy and findings from the verification process and aligns with the bank's risk appetite and policies.

The customer's derivative trade request details are:
[Provide specific details of the trade request here, e.g.,
Underlying Asset: EUR/USD currency pair
Notional Value: $100,000,000
Tenor: 2 years
Trade Type: FX Forward
Client: [Client Name], Credit Rating: [Rating]
Purpose: Hedging foreign exchange exposure]
"""

# Replace bracketed information with actual trade request details
trade_details_prompt = prompt.replace("[Provide specific details of the trade request here...]",
                                       """Underlying Asset: Interest Rate Swap (Fixed-to-Floating USD)
Notional Value: $250,000,000
Tenor: 5 years
Trade Type: Receive Fixed, Pay Floating
Client: [Client Name], Credit Rating: AA-
Purpose: Hedge against rising interest rates on existing floating-rate debt""")

ans = generate_response(trade_details_prompt)
print(ans)


**Initial Assessment:**

**Trade Request Details:**
- **Underlying Asset:** EUR/USD currency pair
- **Notional Value:** $100,000,000
- **Tenor:** 2 years
- **Trade Type:** FX Forward
- **Client:** [Client Name], Credit Rating: [Rating]
- **Purpose:** Hedging foreign exchange exposure

**Potential Benefits:**
1. **Hedging:** The trade provides the client with a hedge against foreign exchange risk, aligning with their stated purpose.
2. **Revenue Generation:** The bank can earn fees and potentially benefit from favorable market movements.
3. **Client Relationship:** Strengthens the relationship with the client by meeting their hedging needs.

**Immediate Red Flags:**
1. **Counterparty Credit Risk:** The client's credit rating needs to be assessed to ensure it falls within acceptable limits.
2. **Regulatory Compliance:** Ensure compliance with Dodd-Frank, EMIR, and other relevant regulations.
3. **Market Conditions:** Verify that the pricing reflects current market conditions and volatili

## ReAct Prompting
**Description:** ReAct (Reasoning and Acting) prompting is a technique that combines reasoning trace generation with task-specific actions. It encourages the model to interleave thinking steps (Reasoning) with taking actions (Acting) through tools or interacting with an environment. The model observes the outcomes of its actions and uses this information to refine its subsequent reasoning and actions. This allows the model to perform tasks that require interacting with external sources of information or performing physical (or simulated physical) actions.

**Application Areas:** Tasks that require external knowledge lookup, interacting with APIs or tools, complex decision-making, and environments where actions have observable consequences. Examples include answering questions that require searching the web, using a calculator for complex math, interacting with a simulated game environment, or completing tasks that involve multiple external steps.

**When to Use:**
- When the task requires up-to-date information not present in the model's training data.
- When the task involves complex calculations or operations that the model is not inherently good at performing reliably.
- When the task requires interacting with external systems or data sources.
- To enable the model to break down a complex task into smaller, executable steps based on external feedback.

**How it Works (Simulated in a Prompt):** In a prompt, ReAct is simulated by guiding the model to explicitly state its "Thought" process (reasoning about the next step or action) and its "Action" (the action it would take or the tool it would use), followed by an "Observation" (the simulated result of the action). The model then uses the Observation to inform its next Thought and Action.


In [36]:
prompt = """
You are a helpful assistant that can answer questions by first thinking step-by-step about the question, then deciding on an action (like searching or using a tool),
observing the result of the action, and then repeating the process until you can formulate a final answer.

Your process should follow this format:

Thought: Think step-by-step about the question and what information you need to answer it.
Action: [Describe the action you would take or tool you would use]
Observation: [Simulate the result of the action]
... (Repeat Thought, Action, Observation until you have enough information)
Thought: Now that I have the necessary information, I can formulate the final answer.
Final Answer: [Provide the final answer]

Here is the question: What is the current population of Tokyo?

"""

ans = generate_response(prompt)
print(ans)


Thought: To find the current population of Tokyo, I need to access the most recent demographic data or statistics. This information is typically available from government databases, statistical agencies, or reputable news sources.

Action: I would search for the latest population statistics for Tokyo from a reliable source such as a government website or a recent report.

Observation: The search reveals that the current population of Tokyo is approximately 14 million people in the 23 special wards, and about 37 million in the Greater Tokyo Area as of the latest data.

Thought: Now that I have the necessary information, I can formulate the final answer.

Final Answer: The current population of Tokyo is approximately 14 million in the 23 special wards and about 37 million in the Greater Tokyo Area.


In [37]:

prompt = """
You are an expert financial analyst assistant. You need to determine if a company's stock is overvalued based on its current P/E ratio compared to the industry average.

Follow the ReAct process:

Thought: Think step-by-step about how to assess if the stock is overvalued based on the P/E ratio.
Action: [Describe the action to find the company's current stock price and EPS to calculate its P/E ratio]
Observation: [Simulate the result, e.g., "Company X Stock Price: $150, EPS: $5.00"]
Thought: Calculate the company's P/E ratio.
Action: [Describe the calculation: Stock Price / EPS]
Observation: [Simulate the result, e.g., "Company X P/E Ratio: 30x"]
Thought: Find the industry average P/E ratio for comparison.
Action: [Describe the action to find the industry average P/E ratio for the relevant sector (e.g., Technology)]
Observation: [Simulate the result, e.g., "Industry Average P/E Ratio (Technology): 25x"]
Thought: Compare the company's P/E ratio to the industry average and determine if it is overvalued.
Action: [Describe the comparison and decision process]
Observation: [Simulate the result of the comparison]
Thought: Formulate the final assessment.
Final Answer: [Provide the final assessment based on the P/E comparison]

Here is the company: Company X (Technology Sector)
"""

ans = generate_response(prompt)
print(ans)

Thought: To determine if Company X's stock is overvalued, I need to compare its P/E ratio to the industry average. First, I need to find the company's current stock price and EPS to calculate its P/E ratio.

Action: Look up Company X's current stock price and EPS.

Observation: Company X Stock Price: $150, EPS: $5.00

Thought: Calculate the company's P/E ratio using the formula: Stock Price / EPS.

Action: Calculate the P/E ratio: $150 / $5.00

Observation: Company X P/E Ratio: 30x

Thought: Find the industry average P/E ratio for the Technology sector for comparison.

Action: Look up the industry average P/E ratio for the Technology sector.

Observation: Industry Average P/E Ratio (Technology): 25x

Thought: Compare Company X's P/E ratio to the industry average to determine if it is overvalued.

Action: Compare 30x (Company X) to 25x (Industry Average).

Observation: Company X's P/E ratio is higher than the industry average.

Thought: Since Company X's P/E ratio is higher than the ind

## Conversational App

In [38]:
# creating a function to get outcome
def generate_response(messages,model='gpt-4o'):
  response = client.responses.create(
      model = model,
      input = messages,
      temperature=0.5
  )
  return response.output_text


In [39]:
messages = [{"role":"system","content":"You are a helpful assistant"},]
while True:
    uinput = input("You: ")
    print("You: ",uinput)
    if uinput.lower() in ['q','quit','exit']:
        break
    messages.append({"role":"user","content":uinput})
    response = generate_response(messages=messages)
    print(f"Bot: {response}")
    messages.append({"role":"assistant","content":response})


You: hi
You:  hi
Bot: Hello! How can I assist you today?
You: I live in Bangalore city
You:  I live in Bangalore city
Bot: That's great! Bangalore is known for its vibrant culture and tech industry. Is there anything specific you'd like to know or discuss about the city?
You: nothing for now, i wanted to understand more about Agentic AI 
You:  nothing for now, i wanted to understand more about Agentic AI 
Bot: Agentic AI refers to artificial intelligence systems designed to operate with a degree of autonomy, making decisions and performing tasks without direct human intervention. These systems are often built to achieve specific goals and can adapt to changing environments. Here are some key aspects:

1. **Autonomy**: Agentic AI can operate independently, making decisions based on pre-set goals and learned experiences.

2. **Goal-Oriented**: These systems are designed to achieve specific objectives, which can range from simple tasks to complex problem-solving.

3. **Adaptability**: The

In [40]:
messages

[{'role': 'system', 'content': 'You are a helpful assistant'},
 {'role': 'user', 'content': 'hi'},
 {'role': 'assistant', 'content': 'Hello! How can I assist you today?'},
 {'role': 'user', 'content': 'I live in Bangalore city'},
 {'role': 'assistant',
  'content': "That's great! Bangalore is known for its vibrant culture and tech industry. Is there anything specific you'd like to know or discuss about the city?"},
 {'role': 'user',
  'content': 'nothing for now, i wanted to understand more about Agentic AI '},
 {'role': 'assistant',
  'content': 'Agentic AI refers to artificial intelligence systems designed to operate with a degree of autonomy, making decisions and performing tasks without direct human intervention. These systems are often built to achieve specific goals and can adapt to changing environments. Here are some key aspects:\n\n1. **Autonomy**: Agentic AI can operate independently, making decisions based on pre-set goals and learned experiences.\n\n2. **Goal-Oriented**: Th

## Thank You